# LabWork#1
## *Anastasia Nikiforova*

In [1]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.
import time # Время

Сначала захотелось все же поиграться с тем, что можно делать с requests. Ностальгия!

Готовый класс - в самом низу этого ноутбука.

Отсюда и до [:-1] - процесс работы. Может даже будет весело <s>не гарантирую</s>

In [10]:
requests.get("https://tproger.ru/news/pypl-october-2018/")

<Response [200]>

In [11]:
%%time
resp = requests.get("https://tproger.ru/news/pypl-october-2018/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

cookies: <RequestsCookieJar[<Cookie __cfduid=d1010be6dbee243c3ed39e0b06220398a1539543863 for .tproger.ru/>]>
time to download: 0:00:00.298509
page encoding UTF-8
Server response:  200
Is everything ok?  True
Page's URL:  https://tproger.ru/news/pypl-october-2018/
Wall time: 320 ms


Сейчас попробуем чего-нибудь вытащить со страницы.

In [8]:
findheaders = re.compile('<h1 class="entry-title" itemprop="headline">(.+)</h1>', re.S) # done!
art_body = re.compile('<p>(.+)</p>', re.S)

# Получает текст страницы.
art=requests.get("https://tproger.ru/news/pypl-october-2018/")
# Находим заголовок.
title = findheaders.findall(art.text)[0]
print(title) #works fine!


Python лидирует в рейтинге PYPL за октябрь 2018 года


Все новости - внутри  простых <\p>
удалить внутри картинки и гиперссылки

In [15]:
bs = BeautifulSoup(resp.text, "html5lib") 

title = bs.h1.text 

text = BeautifulSoup(" ".join([p.text for p in bs.find_all("p")[:-7]]), "html5lib").get_text()

trash = BeautifulSoup(" ".join([p.text for p in bs.find_all("p")[-7:]]), "html5lib").get_text()

print("\n-----\n", title, "\n-----\n", text)
print("\n-----\nHere comes the trash (na-na-na-na):\n", trash, "\n-----")


-----
 Python лидирует в рейтинге PYPL за октябрь 2018 года 
-----
 Опубликованы новые данные ежемесячного рейтинга языков программирования PYPL за октябрь 2018 года. Данные разместил Пьер Карбоннелл (Pierre Carbonnelle). Python занял первое место, а язык Java хоть и находится на втором, но немного потерял позиции. Напомним, что рейтинг PYPL формируется на основе того, как часто искали руководства по тому или иному ЯП. Топ-5 по-прежнему составляют Python, Java, JavaScript, C# и PHP, хотя две последние позиции поменялась местами. Perl и Delphi продолжают падать, а вот Kotlin и Go, наоборот, растут. PHP также начал серьёзно падать в рамках рейтинга.  Для сравнения, в 2017 году ситуация выглядела иначе. Тогда лидером был язык Java, а Python располагался на втором месте. На третьем месте шёл PHP, который на текущий момент потерял пару очков, за ним C# от Microsoft, а на пятом месте располагался JavaScript. На графике приведены данные популярности языков программирования из первой десятки,

***Ух! Сработало, как магия!***

### Добыча ссылок   

Нужно добыть ссылки со страницы.

Здесь хотелось бы выразить благодарность разработчикам сайта, поскольку несмотря на то, что у них нет удобной структуры адреса по дням как в Ленте, каталог новостей на Tproger оформлен в виде https://tproger.ru/category/news/page/3/, при этом на страницах новости разделены по дням! 

Ву-ху!

Сразу видно, что имеешь дело с программистами :)

In [24]:
h2s = BeautifulSoup(requests.get("https://tproger.ru/category/news/").text, "html5lib").find_all("h2")
#print(h2s)
links = [l("a")[0]["href"] for l in h2s]
links   # так красивее, чес принтом

['https://tproger.ru/news/nvidia-rapids/',
 'https://tproger.ru/news/yandex-media-services-memorandum/',
 'https://tproger.ru/news/lyrebird-voice-api/',
 'https://tproger.ru/news/native-load-balancing-gke/',
 'https://tproger.ru/news/facebook-leak/',
 'https://tproger.ru/news/apigee-new-capabilities/',
 'https://tproger.ru/news/neural-net-snakes/',
 'https://tproger.ru/news/activeqa-jeopardy/',
 'https://tproger.ru/news/zdnet-fuchsia-code/',
 'https://tproger.ru/news/unity-visual-search/',
 'https://tproger.ru/news/google-pick-out-voice/',
 'https://tproger.ru/news/google-new-services-cloud-identity/',
 'https://tproger.ru/news/neural-networks-illusions/',
 'https://tproger.ru/news/cloud-storage-dual-region-option/',
 'https://tproger.ru/news/poc-code-edge/',
 'https://tproger.ru/news/google-oboe/',
 'https://tproger.ru/news/essential-ai-smartphone/',
 'https://tproger.ru/news/airfase-vulnerability/',
 'https://tproger.ru/news/token-binding-protocol/',
 'https://tproger.ru/news/common-

Ну все, с одной страница все ссылки получили, значит и со всех собрать - не проблема.

А значит, и корпус статей собрать - как пить дать. (тьфу-тьфу)

In [71]:
# Загрузка статьи по URL.
def getOneArticle(url):
    """ getOneArticle gets the body of an article from Tproger.ru"""
    resp = requests.get(url)
    bs = BeautifulSoup(resp.text, "html5lib") 
    aTitle = bs.h1.text
    anArticle = BeautifulSoup(" ".join([p.text for p in bs.find_all("p")[:-7]]), "html5lib").get_text()
    return "\n----------\n" + aTitle + "\n-----\n" + anArticle

# Загрузка всех статей с одной страницы.
def getPage(url):
    """ Gets all URLs with news from one page. """
    res = ""
    try:
        # Грузим страницу со списком всех статей.
        day = requests.get(url) 
        # Получаем фрагменты с нужными нам адресами статей.
        h2s = BeautifulSoup(day.text, "html5lib").find_all("h2")
        # Получаем все адреса на статьи за день.
        links = [l.find_all("a")[0]["href"] for l in h2s]
        # Загружаем статьи.
        for l in links[:10]:
            res += getOneArticle(l)
            #res += "\n\n*****\n"
            time.sleep(0.3) # Мы ведь этичные хакеры.  - Ага 
    except:
        pass
    return res

In [72]:
r = getPage("https://tproger.ru/category/news/")
print (r)


----------
NVIDIA анонсировала платформу GPU-ускорения для обработки больших массивов данных и машинного обучения
-----
После двух лет разработки NVIDIA анонсировала открытую платформу GPU-ускорения RAPIDS для обработки больших массивов данных и машинного обучения на основе архитектуры CUDA. Использование RAPIDS с алгоритмом машинного обучения XGBoost даёт 50-кратное увеличение производительности по сравнению с системами на базе CPU. Платформа базируется на открытых проектах, таких как Apache Arrow, pandas и scikit-learn. NVIDIA планирует добавить в неё новые библиотеки и возможности машинного обучения, для этого компания сотрудничает с Anaconda, BlazingDB, Databricks, Quansight и другими игроками рынка открытого ПО. NVIDIA интегрировала RAPIDS во фреймворк для анализа и обработки данных Apache Spark, чтобы ускорить его распространение. Открытые библиотеки RAPIDS уже доступны на сайте платформы, в ближайшее время компания собирается добавить контейнеризованные версии в репозитории NVI

Теперь осталось только пройти по всем (ну или например по первым 20 страницам на этот раз) страницам новостей и собрать из них тексты.

И в ООП-формате все оформить (мы же воспитанные программисты)

## Формат: ООП 

*(Хотя многие со мной поспорят о том, что Python и ООП как-то связаны)*

In [74]:
import pymorphy2

In [162]:
class getNewsPaper:
         
    # Конструктор - вызывается при создании объекта и инициализирует его.
    def __init__(self):
        self.articles=[]     # Загруженные статьи.
        self.titles=[]       # Заголовки статей.
        self.dictionaries=[] # Словари для каждой из статей.
        # Создаем и загружаем морфологический словарь.
        self.morph=pymorphy2.MorphAnalyzer()
        
    def getOneArticle(self, url):
        """ getOneArticle gets the body of an article from Tproger.ru """
        one_title = []
        one_article = []
        resp = requests.get(url)
        bs = BeautifulSoup(resp.text, "html5lib") 
        
        self.titles.append(bs.h1.text)
        one_title.append(bs.h1.text)
        
        self.articles.append(BeautifulSoup(" ".join([p.text for p in bs.find_all("p")[:-7]]), "html5lib").get_text().replace("\xa0", " "))
        one_article.append(BeautifulSoup(" ".join([p.text for p in bs.find_all("p")[:-7]]), "html5lib").get_text().replace("\xa0", " "))
        
        return one_title, one_article

    # Загрузка всех статей с одной страницы.
    def getPage(self, url):
        """ Gets all URLs with news from one page. """
        return_page = []
        try:
            # Грузим страницу со списком всех статей.
            day = requests.get(url) 
            # Получаем фрагменты с нужными нам адресами статей.
            h2s = BeautifulSoup(day.text, "html5lib").find_all("h2")
            # Получаем все адреса на статьи за день.
            links = [l.find_all("a")[0]["href"] for l in h2s]
            # Загружаем статьи.
            for l in links[:10]:
                self.articles.append(self.getOneArticle(l))
                return_page.append(self.getOneArticle(l))
                time.sleep(0.3)
        except:
            pass
        
        return return_page

    # Загрузка всех статей за несколько дней.
    def getPeriod(self, last):
        #         self.articles.append(self.getPage("https://tproger.ru/category/news/"))
        #         return_period.append(self.getPage("https://tproger.ru/category/news/"))
        
        #         start = 2  
        #         # Можно было бы и с первой начать, но /news/page/1 переадресует на /news/, лучше обойтись без переадресаций
        period = []
        for i in range(last):
            # last+1, т.к. последний элемент нам тоже нужен
            self.articles.append(self.getPage('https://tproger.ru/category/news/' + str(i)))
            period.append(self.getPage('https://tproger.ru/category/news/' + str(i)))
            
        return period
            
    # Заагрузка топовых новостей за последнюю неделю / за все время        
    def getTop(self, **options):
        '''period in ['week', 'month', 'ever', '100']'''
        return_top = []
        
        if options.get("period") == "week":
            self.articles.append(self.getPage('https://tproger.ru/top/week/'))
            return_top.append(self.getPage('https://tproger.ru/top/week/'))
            
        if options.get("period") == "month":    
            self.articles.append(self.getPage('https://tproger.ru/top/month/'))
            return_top.append(self.getPage('https://tproger.ru/top/month/'))

        if options.get("period") in ["ever", "100"]:
            # Вообще, на сайте предлагают загрузить топовые новости за последние 100 лет. Юмористы
            # но раз предлагают, включим "100"
            self.articles.append(self.getPage('https://tproger.ru/top/ever/'))
            return_top.append(self.getPage('https://tproger.ru/top/ever/'))
            
        else:
            pass
        
        return return_top

    # Потроение вектора для статьи.
    posConv = {'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
    def getArticleDictionary(self, text, needPos = None):
        words = [a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords = []
    
        for w in words:
            wordform = self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    if needPos != None:
                        reswords.append(wordform.normal_form + self.posConv[wordform.tag.POS])
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat = Counter(reswords)
        stat = {a: stat[a] for a in stat.keys() if stat[a] > 1}
        return stat

    # Посчитаем вектора для всех статей.
    def calcArticleDictionaries(self, needPos = None):
        self.dictionaries = []
        for a in self.articles:
            self.dictionaries.append(self.getArticleDictionary(a, needPos))
            
    # Сохраняем статьи в файл.
    def saveArticles(self, filename):
        """ Saves all articles to a file with a filename. """
        newsfile = open(filename, "w")
        for art in self.articles:
            newsfile.write('\n=====\n' + art)
        newsfile.close()

    # Читаем статьи из файла.
    def loadArticles(self, filename):
        """ Loads and replaces all articles from a file with a filename. """
        newsfile = open(filename, encoding="utf-8")
        text = newsfile.read()
        self.articles = text.split('\n=====\n')[1:]
        for a in self.articles:
            self.titles.append(a.split('\n-----\n')[0])
        newsfile.close()

    # Для удобства - поиск статьи по ее заголовку.
    def findNewsByTitle(self, title):
        if title in self.titles:
            return self.titles.index(title)
        else:
            return -1

### Testing

In [148]:
newspaper_getter = getNewsPaper()
a = newspaper_getter.getPage("https://tproger.ru/category/news/")
print(a)

[(['NVIDIA анонсировала платформу GPU-ускорения для обработки больших массивов данных и машинного обучения'], ['После двух лет разработки NVIDIA анонсировала открытую платформу GPU-ускорения RAPIDS для обработки больших массивов данных и машинного обучения на основе архитектуры CUDA. Использование RAPIDS с алгоритмом машинного обучения XGBoost даёт 50-кратное увеличение производительности по сравнению с системами на базе CPU. Платформа базируется на открытых проектах, таких как Apache Arrow, pandas и scikit-learn. NVIDIA планирует добавить в неё новые библиотеки и возможности машинного обучения, для этого компания сотрудничает с Anaconda, BlazingDB, Databricks, Quansight и другими игроками рынка открытого ПО. NVIDIA интегрировала RAPIDS во фреймворк для анализа и обработки данных Apache Spark, чтобы ускорить его распространение. Открытые библиотеки RAPIDS уже доступны на сайте платформы, в ближайшее время компания собирается добавить контейнеризованные версии в репозитории NVIDIA GPU C

In [149]:
test = getNewsPaper()
a = test.getOneArticle("https://tproger.ru/articles/useful-linux-commands/")
print(a)

(['Работаем в терминале Linux как профи: подборка полезных команд'], ['Видели про-юзеров Linux, которые эффективно работают в терминале? Хотите так же? Не вопрос: в этой статье мы собрали некоторые из полезных команд, которые помогут вам сэкономить время и повысить продуктивность. Когда вы начинаете что-то вводить в терминале, вы можете нажать Tab и вам будут предложены возможные варианты продолжения, которые начинаются с введённой вами строки. Например, если вы хотите скопировать файл с именем file1.txt, вы можете ввести только cp f, нажать Tab и увидеть возможные варианты. Также Tab можно использовать для автодополнения команд. Представьте ситуацию, когда вы спустились глубоко по иерархии папок, затем перешли в папку, которая находится в совершенно другом месте, а потом поняли, что вам нужно вернуться обратно. В таком случае вам всего лишь нужно ввести следующую команду: cd - Она вернёт вас в последнюю рабочую директорию и вам не придётся вручную вводить длинный путь. Примечание Посл

In [164]:
test_period = getNewsPaper()
a = test_period.getPeriod(3)
print(a)

[[(['Низкоуровневая модель памяти'], ['В прошлой статье была рассмотрена высокоуровневая часть модели памяти. В этой статье подробно описано, что на самом деле происходит с памятью в компьютере на примере Intel x86_64. Схематично и очень упрощённо модель памяти выглядит так:  Статья детально описывает эту модель, а также принципы работы виртуальной памяти в защищённом режиме. Регистр процессора — блок памяти, расположенный прямо на кристалле процессора и образующий сверхбыструю оперативную память (СОЗУ). В x86_64 размеры регистра, за исключением векторных, варьируются от 8 до 64 бит. Программа может быстро работать только с памятью, находящейся в регистрах процессора. Даже для простого инкремента ячейки памяти из RAM (Random Access Memory) необходимо выполнить 3 операции: сначала загрузить эту ячейку из RAM или кэша в регистр, затем инкрементировать и после выгрузить ячейку из регистра обратно в RAM или кэш. Количество и специализация регистров зависит от архитектуры процессора, но тип

In [165]:
test_top = getNewsPaper()
t = test_top.getTop(period="week")
print(t)

[[(['Что делать, если в наследство достался некачественный код — отвечают эксперты'], ['Программирование — это не только написание кода, но и его исправление. Наш подписчик явно столкнулся с ситуацией, где ему потребовался совет опытного человека: «Что делать, если в наследство достался некачественный код?» За разъяснениями мы обратились к нашим экспертам, а полученные ответы предоставляем вашему вниманию. Если вам в голову пришел такой вопрос, то главное не бежать все переписывать. Обычно это стандартная реакция одного программиста на код другого программиста — в коде ничего непонятно, сложно разбираться. Кажется проще переписать все с нуля. Переписать вы всегда успеете, сначала попробуйте разобраться в требованиях и способах использования приложения. Обычно кодовая база растет эволюционно и накапливает бизнес-логику, которая может быть неочевидной для внешнего человека. Попытка переписать эту логику с нуля породит много багов и проблем. Если после недели чтения кода и документации вс

**ОБОЖЕЭТОРАБОТАЕТ!!!!!**

<s>в кои-то веки!</s>

Домашнее задание выполнено.

В процессе было очень многое повторено, изучено и отработано, за что большое спасибо!